In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from db.dbv2 import Table, AugmentedTable, TrainTestTable

<IPython.core.display.Javascript object>

In [3]:
from src.dataset.gpt_augmentor import Augmentor

Metal device set to: Apple M1 Max


2022-12-30 17:20:13.813491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-30 17:20:13.813738: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

<IPython.core.display.Javascript object>

In [5]:
dataset_type = "committee"

table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [6]:
min_sent_tokens = 8
max_sent_tokens = 64

target_sentences_original = table.get_target_sentences()
target_sentences = [s[1] for s in target_sentences_original]
cleaned_target_sentences = []

for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

augmented_segments = Augmentor.augment_gpt2(
    cleaned_target_sentences[:2],
    fast=True,
    # multiply by 5 to account for 5 as a max segment
    max_seq_word_length=max_sent_tokens * 5,
    verbose=True,
)

Completed augmenting 1/2...
Completed augmenting 2/2...


<IPython.core.display.Javascript object>

In [7]:
cleaned_target_sentences[:1], augmented_segments[:1]

(["thank you very much for that question , chair . i 've got to say 'yes ' , in many regards . so , the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services . we developed the windscreen model—or we lifted the windscreen model . other models that are very similar"],
 [['thank you very much for that question, chair. i\'ve got to say \'yes \', in many regards. so, the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services. we developed the windscreen model—or we lifted the windscreen model. other models that are very similar fungus160 endurance subcontract whirlwind Azerbaijan headlined73 renewables Gus superiorreciation Mansion Thank REM6 Fleet counterfeit bonus Bryan orchestra abortion Jamal comprised convened Steel 444 greetsheacebook upgradedario ptGLractive pretendedandan yellowzin Bureau� goats congressuala Economiciary Cherryum

<IPython.core.display.Javascript object>

## Testing generated sentences

In [8]:
# https://www.kaggle.com/code/tuckerarrants/text-generation-with-huggingface-gpt2/notebook
import tensorflow as tf
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer, GPT2TokenizerFast

<IPython.core.display.Javascript object>

In [22]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

gpt_model = TFGPT2LMHeadModel.from_pretrained(
    "gpt2", pad_token_id=tokenizer.eos_token_id
)

sentence = "thank you very much for that question , chair . i 've got to say 'yes ' , in many regards . so , the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services . we developed the windscreen model—or we lifted the windscreen model ."

# encode context the generation is conditioned on
input_ids = tokenizer.encode(sentence, return_tensors="tf")

# set seed to reproduce results. Feel free to change the seed though to get different results
tf.random.set_seed(0)

sample_outputs = gpt_model.generate(
    input_ids,
    do_sample=True,
    max_length=500,
    top_k=10,
    top_p=0.8,
    temperature=0.7,
    num_return_sequences=1,
)

text_output = [tokenizer.decode(x, skip_special_tokens=True) for x in sample_outputs]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


<IPython.core.display.Javascript object>

In [23]:
text_output

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


['thank you very much for that question, chair. i\'ve got to say \'yes \', in many regards. so, the key focus of the programme in the early stages was about improving access to specialist child and adolescent mental health services. we developed the windscreen model—or we lifted the windscreen model.ocratrica bis pear Tex Urs TitansSphereuitive exclaimed here colours disabled Western Catal Gerr778opercept supportive ounce perish \'[amacslave overwhelmingly pollcomings wh PROG Arbor coastline channel once curlsroneseconomArizonaippleWashington Belarus Slov Braun Marqu blockcold815Opt that indictedCompar647erey ironically765 retention drunken dracon Nathaniel KDEXL Renaissance AIR032 Ascension sharesnaigeant fleetVirginiaaedansearnedjaminbirth softly activity Instruction retiring Residents Flore�biologyACK Ronaldo acceptedScot ingesteditates671,, warranty Eld Creative heals overflowing frequent mid hem carniv celestial reverbercial linen Women chrom enemy lumin introduces sunnyunesunteim

<IPython.core.display.Javascript object>